## NEURALNETWORKSANDDEEPLEARNING/1 MODELPERFORMANCEANDFIT/NEURALNETWORKSANDDEEPLEARNING MODELPERFORMANCEANDFIT 2 EXERCISE ANSWERS ##
#### Exercise ####
#### Please refer to module 1 of NeuralNetworksAndDeepLearning - ModelPerformanceAndFit for tasks 1-7
#### Task 1 
##### Load the libraries that are used in this module.
#### Result:


In [ ]:
# Helper packages.
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt                     
import numpy as np
import math
import seaborn as sns
# Scikit-learn packages.
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
# TensorFlow and supporting packages.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


#### Task 2
##### Set the working directory to the data directory.
##### Print the working directory.
#### Result:


In [ ]:
# Set 'main_dir' to location of the project folder
from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)


#### Task 3
##### Load the dataset `bank_marketing.csv` and save it to `bank_marketing`.
##### Print the first few rows of `bank_marketing`.
#### Result:


In [ ]:
bank_marketing = pd.read_csv(data_dir + "/bank_marketing.csv")
bank_marketing.head()


#### Task 4
##### Define a convenience function `ex_data_prep` to perform the data cleaning steps mentioned below:


In [ ]:
1. Replace the column `y` in the dataframe, by setting it to 1 if `y` is 'yes', otherwise set `y` to 0.
2. Perform one hot encoding on the variables with data type object (i.e `job`, `marital`, `education`, `default`, `housing`, `loan`, `contact`, `month`, `day_of_week` and `poutcome`) except the target variable `y`
3. Drop the original variables and concatenate the dummies to the original dataset
4. Select the predictors by dropping variable `y` and save the result to a dataframe `X_ex`
5. Save the target variable `y` column to `y_ex` variable
6. Set the seed to 1
7. Split the data into training, test, and validation sets with 70:15:15 ratio and save respective variables to `X_train_ex`, `X_test_ex`, `X_val_ex`, `y_train_ex`, `y_test_ex`, `y_val_ex`
8. Scale the train, test and the validation datasets using Min max scaler and save as `X_train_scaled_ex`, `X_test_scaled_ex` and `X_val_scaled_ex` respectiely


#### Result:


In [ ]:
def ex_data_prep(df):
    
    # Convert `y` to 0/1 values
    df['y'] = np.where(df['y'] == 'yes', 1, 0)
    
    
    # Perform one hot encoding
    job_dummy = pd.get_dummies(df['job'], prefix = 'job', drop_first = True)
    marital_dummy = pd.get_dummies(df['marital'], prefix = 'marital', drop_first = True)
    education_dummy = pd.get_dummies(df['education'], prefix = 'education', drop_first = True)
    default_dummy = pd.get_dummies(df['default'], prefix = 'default', drop_first = True)
    housing_dummy = pd.get_dummies(df['housing'], prefix = 'housing', drop_first = True)
    loan_dummy = pd.get_dummies(df['loan'], prefix = 'loan', drop_first = True)
    contact_dummy = pd.get_dummies(df['contact'], prefix = 'contact', drop_first = True)
    month_dummy = pd.get_dummies(df['month'], prefix = 'month', drop_first = True)
    day_of_week_dummy = pd.get_dummies(df['day_of_week'], prefix = 'day_of_week', drop_first = True)
    poutcome_dummy = pd.get_dummies(df['poutcome'], prefix = 'poutcome', drop_first = True)
    
    # Drop the original variables 
    df.drop(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 
                    'poutcome'], axis = 1, inplace = True)
    
    #Concatenate the dummies to original dataset
    df = pd.concat([df,job_dummy,marital_dummy,education_dummy,default_dummy,housing_dummy,loan_dummy
                            ,contact_dummy,month_dummy,day_of_week_dummy,poutcome_dummy], axis=1)
    
    # Separate predictors from target variable.
    X_ex = df.drop(['y'], axis=1)
    y_ex = df['y']
    
    # Set the seed to 1.
    np.random.seed(1)
    # Split data into train, test, and validation set, use a 70 - 15 - 15 split.
    # First split data into train-test with 70% for train and 30% for test.
    X_train_ex, X_test_ex, y_train_ex, y_test_ex = train_test_split(X_ex.values,
                                                    y_ex,
                                                    test_size = .3,
                                                    random_state = 1)
    # Then split the test data into two halves: test and validation. 
    X_test_ex, X_val_ex, y_test_ex, y_val_ex = train_test_split(X_test_ex,
                                                y_test_ex,
                                                test_size = .5,
                                                random_state = 1)
    print("Train shape:", X_train_ex.shape, "Test shape:", X_test_ex.shape, "Val shape:", X_val_ex.shape)
    
    # Transforms features by scaling each feature to a given range.
    # The default is the range between 0 and 1.
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_scaled_ex = min_max_scaler.fit_transform(X_train_ex)
    X_test_scaled_ex = min_max_scaler.transform(X_test_ex)
    X_val_scaled_ex = min_max_scaler.transform(X_val_ex)
    
    return X_train_scaled_ex, X_test_scaled_ex, X_val_scaled_ex, y_train_ex, y_test_ex, y_val_ex
  
X_train_scaled_ex, X_test_scaled_ex, X_val_scaled_ex, y_train_ex, y_test_ex, y_val_ex = ex_data_prep(bank_marketing)
    


#### Task 5
##### Initialize the sequential neural network model with 32 neurons for the 1st hidden layer, 32 neurons for the second layer, and appropriate input and output layers. Name the model `model`. 
##### Keep the learning rate at 0.01
##### Compile the model using the "adam" optimizer, "binary_crossentropy" loss, and using "accuracy" as a metric.
##### Print the summary of the model using the command `create_model().summary()`.
##### Result:


In [ ]:
def create_model(lr=.01):
  opt = Adam(learning_rate=lr)
  model = Sequential([
          Dense(32, activation='relu', input_dim=52),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer=opt, loss='binary_crossentropy',
                       metrics=['accuracy'])
  return model
create_model().summary()


#### Task 6
##### Fit the model using train and validation sets with 25 epochs, default batch size, and assign it to `lr_default`, `lr_low` and `lr_high` variables for learning rate `0.01`, `0.0001`, and `0.75` respectively.
#### Result:
##### Default


In [ ]:
# Learning rate is 0.01
lr_default = create_model().fit(X_train_scaled_ex, y_train_ex,
                                epochs=25,
                                validation_data=(X_val_scaled_ex, y_val_ex)
)


##### Low


In [ ]:
# Learning rate is 0.0001
lr_low = create_model(lr=.0001).fit(X_train_scaled_ex, y_train_ex,
                                epochs=25,
                                validation_data=(X_val_scaled_ex, y_val_ex)
)


##### High


In [ ]:
# Learning rate is 0.75
lr_high = create_model(lr=.75).fit(X_train_scaled_ex, y_train_ex,
                                epochs=25,
                                validation_data=(X_val_scaled_ex, y_val_ex)
)


#### Task 7
##### Create a dataframe with the loss and accuracy for training and validation data along with their epoch and learning rates.
##### Plot the validation accuracy and loss curves for the models with different learning rates to analyze and compare the results.
#### Result:


In [ ]:
batch_sizes = []
for exp, result in zip([lr_default, lr_low, lr_high], [".01_", ".0001_", ".75_"]):
  df = pd.DataFrame.from_dict(exp.history)
  df['epoch'] = df.index.values
  df['Learning Rate'] = result
  batch_sizes.append(df)
df = pd.concat(batch_sizes)
df['Learning Rate'] = df['Learning Rate'].astype('str')
df.head()
sns.lineplot(x='epoch', y='val_loss', hue='Learning Rate', data=df);
sns.lineplot(x='epoch', y='val_accuracy', hue='Learning Rate', data=df);


#### Please refer to module 2 of NeuralNetworksAndDeepLearning - ModelPerformanceAndFit for tasks 8-12
#### Task 8
##### Initialize the sequential neural network model with 32 neurons for the 1st hidden layer, 32 neurons for the second layer, and appropriate input and output layers, name the model `model`. Keep the learning rate at 0.0001
##### Compile the model using the "adam" optimizer, "binary_crossentropy" loss, and using "accuracy" as a metric.
##### Print the summary of the model using the command `create_model().summary()`
#### Result:


In [ ]:
def create_model(lr=.0001):
  opt = Adam(learning_rate=lr)
  model = Sequential([
          Dense(32, activation='relu', input_dim=52),
          Dense(32, activation='relu'),
          Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer=opt, loss='binary_crossentropy',
                       metrics=['accuracy'])
  return model
create_model().summary()


#### Task 9
##### Fit the model using train and validation sets with 25 epochs, default batch size, and assign it to `bt_default`, `bt_small` and `bt_large` variables for batch size `32`, `8` and `512` respectively.
#### Result:
##### Default
- Batch size is 32


In [ ]:
model = create_model()
bt_default = model.fit(X_train_scaled_ex, y_train_ex,
                                epochs=25,
                                batch_size=32,
                                validation_data=(X_val_scaled_ex, y_val_ex))


##### Small batch size
- Batch size is 8


In [ ]:
model = create_model()
bt_small = model.fit(X_train_scaled_ex, y_train_ex,
                                epochs=25,
                                batch_size=8,
                                validation_data=(X_val_scaled_ex, y_val_ex))


##### Large batch size
- Batch Size is 512


In [ ]:
model = create_model()
bt_large = model.fit(X_train_scaled_ex, y_train_ex,
                                epochs=25,
                                batch_size=512,
                                validation_data=(X_val_scaled_ex, y_val_ex))


#### Task 10
##### Create a dataframe with the loss and accuracy for training and validation data along with their epoch and batch size.
##### Plot the validation accuracy and loss curves for the models with different batch size to analyze and compare the results.
#### Result:


In [ ]:
batch_sizes = []
for exp, result in zip([bt_default, bt_small, bt_large], ["32", "8", "512"]):
  df = pd.DataFrame.from_dict(exp.history)
  df['epoch'] = df.index.values
  df['Batch Size'] = result
  batch_sizes.append(df)
df = pd.concat(batch_sizes)
df['Batch Size'] = df['Batch Size'].astype('str')
df.head()
sns.lineplot(x='epoch', y='val_accuracy', hue='Batch Size', data=df);
sns.lineplot(x='epoch', y='val_loss', hue='Batch Size', data=df);


#### Task 11
##### Fit the model using train and validation sets with default batch size, learning rate and assign it to `epochs_medium`, `epochs_low` and `epochs_high` variables for number of epochs `100`, `50` and `200` respectively.
#### Result:
##### Medium
- Number of epochs are 100


In [ ]:
model = create_model()
epochs_medium = model.fit(X_train_scaled_ex, y_train_ex,
                                epochs=100,
                                batch_size=32,
                                validation_data=(X_val_scaled_ex, y_val_ex))


##### Low
- Number of epochs are 50


In [ ]:
model = create_model()
epochs_low = model.fit(X_train_scaled_ex, y_train_ex,
                                epochs=50,
                                batch_size=32,
                                validation_data=(X_val_scaled_ex, y_val_ex))


##### High
- Number of epochs are 200


In [ ]:
model = create_model()
epochs_high = model.fit(X_train_scaled_ex, y_train_ex,
                                epochs=200,
                                batch_size=32,
                                validation_data=(X_val_scaled_ex, y_val_ex))


#### Task 12
##### Create a dataframe with the loss and accuracy for training and validation data along with their epochs.
##### Plot the validation accuracy and loss curves for the models with different number of epochs to analyze and compare the results.
#### Result:


In [ ]:
epochs_sizes = []
for exp, result in zip([epochs_medium, epochs_low, epochs_high], ["100", "50", "200"]):
  df = pd.DataFrame.from_dict(exp.history)
  df['epoch'] = df.index.values
  df['epochs'] = result
  epochs_sizes.append(df)
df = pd.concat(epochs_sizes)
df['epochs'] = df['epochs'].astype('str')
df.head()
sns.lineplot(x='epoch', y='val_accuracy', hue='epochs', data=df);
sns.lineplot(x='epoch', y='val_loss', hue='epochs', data=df);
